In [1]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk

In [2]:

df_normal = pd.read_excel(r"C:\Users\VIKRAM\Downloads\NLP DATASET.xlsx")
df_diseased = pd.read_excel(r"C:\Users\VIKRAM\Downloads\nlp dieseases dataset.xlsx")


In [3]:
df_normal.head()

,type,meal type,person type,diet type,recommend
0,veg,breakfast,atheletic,high protein,"panner salad with some brown bread,add some oa..."
1,non veg,breakfast,normal,normal,idly or dosa or any type of breakfast
2,veg,breakfast,weight loss,low carb,"fruits and panner,sugar free foods"
3,non veg,breakfast,weight gain,high protein,eggs and chicken salad and some beetroot juice
4,non veg,lunch,atheletic,high protein,chicken breast and some vegetables like capsic...


In [4]:
df_normal.isnull().sum()

type           0
meal type      0
person type    0
diet type      0
recommend      0
dtype: int64

In [5]:
df_diseased.head()

,type,meal type,disease type,recommend
0,veg,breakfast,fever,Idli with ginger chutney; warm lemon water
1,non veg,breakfast,fever,Masala omelette with toast; ginger tea.
2,veg,lunch,fever,Vegetable khichdi; light vegetable soup
3,non veg,lunch,fever,Chicken soup; boiled rice with steamed vegetab...
4,veg,dinner,fever,Palak dal with rice; warm turmeric milk.


In [6]:
df_diseased.isnull().sum()

type            0
meal type       0
disease type    0
recommend       0
dtype: int64

In [7]:

def preprocess_text(text):
   
    text = text.lower()
    
   
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    
    
    words = word_tokenize(text)
    
 
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]
    
   
    return ' '.join(words)


In [8]:

df_normal['type'] = df_normal['type'].apply(preprocess_text)
df_normal['meal type'] = df_normal['meal type'].apply(preprocess_text)
df_normal['person type'] = df_normal['person type'].apply(preprocess_text)
df_normal['diet type'] = df_normal['diet type'].apply(preprocess_text)

In [9]:
df_diseased['type'] = df_diseased['type'].apply(preprocess_text)
df_diseased['meal type'] = df_diseased['meal type'].apply(preprocess_text)
df_diseased['disease type'] = df_diseased['disease type'].apply(preprocess_text)

In [10]:
df_normal['features'] = df_normal['type'] + " " + df_normal['meal type'] + " " + df_normal['person type'] + " " + df_normal['diet type']
df_diseased['features'] = df_diseased['type'] + " " + df_diseased['meal type'] + " " + df_diseased['disease type']

In [11]:
df_normal['features']

0           veg breakfast atheletic high protein
1                non veg breakfast normal normal
2             veg breakfast weight loss low carb
3     non veg breakfast weight gain high protein
4           non veg lunch atheletic high protein
5                    non veg lunch normal normal
6             non veg lunch weight loss low carb
7            non veg lunch weight gain high carb
8              veg dinner atheletic high protein
9                   non veg dinner normal normal
10               veg dinner weight loss low carb
11       non veg dinner weight gain high protein
12                veg breakfast atheletic normal
13                   veg breakfast normal normal
14        non veg breakfast weight loss low carb
15        non veg lunch weight gain high protein
16          non veg lunch atheletic high protein
17                   non veg lunch normal normal
18               veg dinner weight loss low carb
19       non veg dinner weight gain high protein
Name: features, dtyp

In [12]:
df_diseased['features']

0         veg breakfast fever
1     non veg breakfast fever
2             veg lunch fever
3         non veg lunch fever
4            veg dinner fever
               ...           
73       veg breakfast dengue
74        nonveg lunch dengue
75           veg lunch dengue
76      non veg dinner dengue
77          veg dinner dengue
Name: features, Length: 78, dtype: object

In [13]:

vectorizer_normal = TfidfVectorizer()
vectorizer_diseased = TfidfVectorizer()

tfidf_matrix_normal = vectorizer_normal.fit_transform(df_normal['features'])
tfidf_matrix_diseased = vectorizer_diseased.fit_transform(df_diseased['features'])


In [14]:
tfidf_matrix_normal

<20x14 sparse matrix of type '<class 'numpy.float64'>'
	with 112 stored elements in Compressed Sparse Row format>

In [15]:
tfidf_matrix_diseased

<78x21 sparse matrix of type '<class 'numpy.float64'>'
	with 272 stored elements in Compressed Sparse Row format>

In [16]:
def recommend_food_nlp(diet_preference, **kwargs):
    if diet_preference == "normal":
      
        food_type = preprocess_text(kwargs.get('food_type'))
        meal_type = preprocess_text(kwargs.get('meal_type'))
        person_type = preprocess_text(kwargs.get('person_type'))
        diet_type = preprocess_text(kwargs.get('diet_type'))
        user_input = f"{food_type} {meal_type} {person_type} {diet_type}"
        
       
        user_tfidf = vectorizer_normal.transform([user_input])
        
        similarities = cosine_similarity(user_tfidf, tfidf_matrix_normal)
        idx = similarities.argsort()[0][-1]
        
        
        return df_normal['recommend'].iloc[idx]
    
    elif diet_preference == "diseased":
      
        type_ = preprocess_text(kwargs.get('type'))
        meal_type = preprocess_text(kwargs.get('meal_type'))
        disease_type = preprocess_text(kwargs.get('disease_type'))
        user_input = f"{type_} {meal_type} {disease_type}"
        
    
        user_tfidf = vectorizer_diseased.transform([user_input])
        
  
        similarities = cosine_similarity(user_tfidf, tfidf_matrix_diseased)
        idx = similarities.argsort()[0][-1]
        
        
        return df_diseased['recommend'].iloc[idx]
    
    else:
        return "Invalid diet preference. Choose 'normal' or 'diseased'."


In [17]:

diet_preference = input("Would you like a 'normal' or 'diseased' diet recommendation? ").strip().lower()

if diet_preference == "normal":
    meal_type = input("Enter the meal type (e.g., breakfast, lunch, dinner): ").strip().lower()
    food_type = input("Enter the food type (e.g., veg, non-veg): ").strip().lower()
    person_type = input("Enter the person type (e.g., athletic, elderly, etc.): ").strip().lower()
    diet_type = input("Enter the diet type (e.g., high protein, low carb, etc.): ").strip().lower()
    

    recommendation = recommend_food_nlp(diet_preference, meal_type=meal_type, food_type=food_type, person_type=person_type, diet_type=diet_type)
    print("Recommended food:", recommendation)

elif diet_preference == "diseased":
    type_ = input("Enter the food type (e.g., veg, non-veg): ").strip().lower()
    meal_type = input("Enter the meal type (e.g., breakfast, lunch, dinner): ").strip().lower()
    disease_type = input("Enter the disease type (e.g., diabetes, hypertension): ").strip().lower()
    
 
    recommendation = recommend_food_nlp(diet_preference, type=type_, meal_type=meal_type, disease_type=disease_type)
    print("Recommended food:", recommendation)
    
else:
    print("Invalid input. Please restart and enter 'normal' or 'diseased' for diet preference.")


Recommended food: eggs and chicken salad and some beetroot juice


BASIC GUI REPRESENTATION OF OUR PROJECT

In [1]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import tkinter as tk
from tkinter import ttk, messagebox

df_normal = pd.read_excel(r"C:\Users\VIKRAM\Downloads\NLP DATASET.xlsx")
df_diseased = pd.read_excel(r"C:\Users\VIKRAM\Downloads\nlp dieseases dataset.xlsx")


def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    words = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]
    return ' '.join(words)


df_normal['features'] = df_normal['type'].apply(preprocess_text) + " " + df_normal['meal type'].apply(preprocess_text) + " " + \
                        df_normal['person type'].apply(preprocess_text) + " " + df_normal['diet type'].apply(preprocess_text)
df_diseased['features'] = df_diseased['type'].apply(preprocess_text) + " " + df_diseased['meal type'].apply(preprocess_text) + " " + \
                          df_diseased['disease type'].apply(preprocess_text)


vectorizer_normal = TfidfVectorizer()
vectorizer_diseased = TfidfVectorizer()
tfidf_matrix_normal = vectorizer_normal.fit_transform(df_normal['features'])
tfidf_matrix_diseased = vectorizer_diseased.fit_transform(df_diseased['features'])


def recommend_food_nlp(diet_preference, **kwargs):
    if diet_preference == "normal":
        user_input = f"{preprocess_text(kwargs.get('food_type'))} {preprocess_text(kwargs.get('meal_type'))} {preprocess_text(kwargs.get('person_type'))} {preprocess_text(kwargs.get('diet_type'))}"
        user_tfidf = vectorizer_normal.transform([user_input])
        similarities = cosine_similarity(user_tfidf, tfidf_matrix_normal)
        idx = similarities.argsort()[0][-1]
        return df_normal['recommend'].iloc[idx]
    
    elif diet_preference == "diseased":
        user_input = f"{preprocess_text(kwargs.get('type'))} {preprocess_text(kwargs.get('meal_type'))} {preprocess_text(kwargs.get('disease_type'))}"
        user_tfidf = vectorizer_diseased.transform([user_input])
        similarities = cosine_similarity(user_tfidf, tfidf_matrix_diseased)
        idx = similarities.argsort()[0][-1]
        return df_diseased['recommend'].iloc[idx]


def on_submit():
    diet_pref = diet_preference.get()
    if diet_pref == "normal":
        recommendation = recommend_food_nlp(
            diet_pref,
            meal_type=meal_type_entry.get(),
            food_type=food_type_entry.get(),
            person_type=person_type_entry.get(),
            diet_type=diet_type_entry.get()
        )
    elif diet_pref == "diseased":
        recommendation = recommend_food_nlp(
            diet_pref,
            type=food_type_entry.get(),
            meal_type=meal_type_entry.get(),
            disease_type=disease_type_entry.get()
        )
    else:
        messagebox.showerror("Error", "Invalid diet preference selected.")
        return
    
    messagebox.showinfo("Recommended Food", recommendation)

def update_fields(*args):
    if diet_preference.get() == "normal":
        person_type_label.grid(row=3, column=0, padx=10, pady=5)
        person_type_entry.grid(row=3, column=1, padx=10, pady=5)
        diet_type_label.grid(row=4, column=0, padx=10, pady=5)
        diet_type_entry.grid(row=4, column=1, padx=10, pady=5)
        disease_type_label.grid_forget()
        disease_type_entry.grid_forget()
    elif diet_preference.get() == "diseased":
        person_type_label.grid_forget()
        person_type_entry.grid_forget()
        diet_type_label.grid_forget()
        diet_type_entry.grid_forget()
        disease_type_label.grid(row=3, column=0, padx=10, pady=5)
        disease_type_entry.grid(row=3, column=1, padx=10, pady=5)

root = tk.Tk()
root.title("Diet Recommendation System")


diet_preference = tk.StringVar()
diet_preference.trace('w', update_fields)
diet_label = ttk.Label(root, text="Diet Preference:")
diet_label.grid(row=0, column=0, padx=10, pady=5)
diet_menu = ttk.Combobox(root, textvariable=diet_preference, values=["normal", "diseased"], state="readonly")
diet_menu.grid(row=0, column=1, padx=10, pady=5)

meal_type_label = ttk.Label(root, text="Meal Type:")
meal_type_label.grid(row=1, column=0, padx=10, pady=5)
meal_type_entry = ttk.Entry(root)
meal_type_entry.grid(row=1, column=1, padx=10, pady=5)

food_type_label = ttk.Label(root, text="Food Type:")
food_type_label.grid(row=2, column=0, padx=10, pady=5)
food_type_entry = ttk.Entry(root)
food_type_entry.grid(row=2, column=1, padx=10, pady=5)

person_type_label = ttk.Label(root, text="Person Type:")
person_type_entry = ttk.Entry(root)
diet_type_label = ttk.Label(root, text="Diet Type:")
diet_type_entry = ttk.Entry(root)

disease_type_label = ttk.Label(root, text="Disease Type:")
disease_type_entry = ttk.Entry(root)

submit_button = ttk.Button(root, text="Get Recommendation", command=on_submit)
submit_button.grid(row=5, column=0, columnspan=2, pady=10)

update_fields()
root.mainloop()
